1. Scrape relevant data off desired sites 
- 99.co v/
- srx or propertyguru

2. Create a CLI code 

3. You want a daily run-able code to check the housing prices
- Weekly reminder in Calendar for now
- Think of a way to automate this or deploy this or control this using more convenient means

4. Check for proximity v/
- Calculate distance from home. v/
- 4km check

5. Final payable price/ loan duration / 

6. Save into a csv v/

7. Telegram bot 

8. Geo-Visualization

Web Scraping 

In [1]:
from datetime import date
today = date.today()
today = today.strftime("%d_%b")

In [2]:
filename="flats"+today+".csv"
f= open(filename, "w")
headers= "Address,District,Rooms,Lease,Cost,Psf,ft2\n"
f.write(headers)

42

In [3]:
from bs4 import BeautifulSoup as soup
import requests 
from urllib.request import urlopen
import sys 
import re 
import time 

In [4]:
def ninenineco(url):
    r = requests.get(url = url)
    if r.status_code == 200:
        print("Access successful")
    else:
        print("Unable to fetch.")
        sys.exit()
    uclient= urlopen(url)
    page_html=uclient.read()
    uclient.close()
    page_soup= soup(page_html, "html.parser")
    counter = 0
    listings=page_soup.find_all("div", {"class":"_2J3pS _3AXiL"})

    for listing in listings:  
        address = listing.find("h5",{"class":"_3Ajbv _30I97 _1vzK2"}).text
        num_rooms = listing.find_all("p",{"class":"_2y86Q _1LPAx _2rhE-"})
        # District, Rooms, Years left
        def dry(info):
            regex_d = re.compile(r'>(S.*?)<')
            regex_r = re.compile(r'>(HDB.*?)<')
            regex_y = re.compile(r'>((\d{4})?\s·\s(\d\d\syears)?)<')
            regex_matrix = [regex_d, regex_r, regex_y]
            dry = [] 
            for i in range(3):
                match = regex_matrix[i].search(info)
                if match == None:
                    dry.append("None")
                else:
                    dry.append(match.group(1))

            return dry[0], dry[1], dry[2]
        district, rooms, years = dry(str(num_rooms))

        # Size and Price: sp 
        sp = listing.find("div", {"class":"bqekS h0kiA"})
        price = sp.find("p",{"class":"_2sIc2 JlU_W _2rhE-"}).text
        price = price.replace(",","")
        price = price.replace("$","")
        size = sp.find_all("p",{"class":"dniCg _2rhE- _1c-pJ"})

        # Per square feet, Square feet
        def ps(info):
            regex_psf = re.compile(r'\$(\d,)?(\d){3}\spsf')
            regex_sqft = re.compile(r'(\d,)?(\d){3}\ssqft')
            regex_matrix = [regex_psf, regex_sqft]
            ps = [] 
            for i in range(2):
                match = regex_matrix[i].search(info)
                if match == None:
                    ps.append("None")
                else:
                    matched = match.group()
                    matched = matched.replace(",","")
                    ps.append(matched)          
            return ps[0], ps[1]
        psf, sqft = ps(str(size))
        f.write(address + ',' + district + ',' + rooms + ',' + years + ',' + price + ',' + psf + ',' + sqft + '\n')
        counter += 1 
        print("Collected " + str(counter)+ " listings")

In [5]:
num_of_pages = int(input("Number of pages to scrape: "))
def search(num_of_pages):
    for i in range(1,num_of_pages+1):
#         url = "https://www.99.co/singapore/sale?listing_type=sale&main_category=hdb&map_bounds=1.276207%2C103.787241%2C1.47917%2C104.000649&page_num="+str(i)+"&page_size=35&property_segments=residential&query_coords=1.3039947%2C103.8298507&query_ids=dtdistrict19&query_limit=radius&query_type=district&radius_max=1000&rental_type=unit&show_cluster_preview=true&show_future_mrts=true&sub_categories=hdb_2r%2Chdb_3r%2Chdb_4r%2Chdb_5r%2Chdb_executive&zoom=12"
        url = "https://www.99.co/singapore/sale?listing_type=sale&main_category=hdb&map_bounds=1.327878%2C103.84953%2C1.392265%2C103.916361&name=D19&page_num="+str(i)+"&page_size=35&price_max=500000&property_segments=residential&query_coords=1.370676%2C103.889197&query_ids=sg-mrt-hougang%2Csg-mrt-kovan%2Csg-mrt-serangoon&query_limit=radius&query_type=subway_station&radius_max=1000&rental_type=unit&show_cluster_preview=true&show_future_mrts=true&show_future_mrts=true&sub_categories=hdb_4r%2Chdb_5r&zoom=14"
        ninenineco(url)
        time.sleep(5)
        if i == num_of_pages:
            print(str(num_of_pages) + " pages completed")
            print("Closing csv file")
            f.close()

Number of pages to scrape: 3


In [6]:
search(num_of_pages)

Access successful
Collected 1 listings
Collected 2 listings
Collected 3 listings
Collected 4 listings
Collected 5 listings
Collected 6 listings
Collected 7 listings
Collected 8 listings
Collected 9 listings
Collected 10 listings
Collected 11 listings
Collected 12 listings
Collected 13 listings
Collected 14 listings
Collected 15 listings
Collected 16 listings
Collected 17 listings
Collected 18 listings
Collected 19 listings
Collected 20 listings
Collected 21 listings
Collected 22 listings
Collected 23 listings
Collected 24 listings
Collected 25 listings
Collected 26 listings
Collected 27 listings
Collected 28 listings
Collected 29 listings
Collected 30 listings
Collected 31 listings
Collected 32 listings
Collected 33 listings
Collected 34 listings
Collected 35 listings
Access successful
Collected 1 listings
Collected 2 listings
Collected 3 listings
Collected 4 listings
Collected 5 listings
Collected 6 listings
Collected 7 listings
Collected 8 listings
Collected 9 listings
Collected 10 l

Pre processing of Data 
- Proximity Checks 

In [7]:
import json 
# Latitude and Longitude of house https://docs.onemap.sg/#reverse-geocode-svy21
# lat and lon return in units of degrees
def latlon(postal_code):
    query ='https://developers.onemap.sg/commonapi/search?searchVal='+str(postal_code)+'&returnGeom=Y&getAddrDetails=Y'

    resp = requests.get(query)
    data = json.loads(resp.content)
    lat = float(data['results'][0]['LATITUDE'])
    lon = float(data['results'][0]['LONGITUDE'])
    return lat, lon 

In [8]:
# https://www.movable-type.co.uk/scripts/latlong.html
from math import sin, cos, sqrt, atan2, pi

def haversine(lat_target, lon_target, lat_home=1.375584516, lon_home=103.8835163):
    earth_radius = 6373 # km 
    
    lat_home_rad = lat_home*(pi/180) # radians
    lat_target_rad = lat_target*(pi/180) # radians
    
    del_phi = (lat_target-lat_home)*(pi/180) # radians
    del_lambda = (lon_target-lon_home)*(pi/180) # radians
    
    a = (sin(del_phi/2))**2 + cos(lat_home_rad)*cos(lat_target_rad)* ((sin(del_lambda/2))**2)
    
    c = 2* atan2(sqrt(a),sqrt(1-a))
    
    d = earth_radius*c
    
    return d 

Geo Visualization 

In [9]:
import pandas as pd 
import re 
from folium.plugins import HeatMap

In [10]:
df = pd.read_csv('flats.csv')
df = pd.DataFrame(df)
print(df.head(5))
print(df[" District"])

                         Address                District          Rooms  \
0  475B Upper Serangoon Crescent  Singapore 532475 · D19    HDB 4 Rooms   
1        298A Compassvale Street  Singapore 541298 · D19  HDB Executive   
2             641B Punggol Drive  Singapore 822641 · D19    HDB 5 Rooms   
3            135 Edgedale Plains  Singapore 820135 · D19    HDB 5 Rooms   
4               312C Sumang Link  Singapore 823312 · D19    HDB 5 Rooms   

             Lease     Cost        Psf        ft2      
0       · 99 years  $675000   $682 psf   990 sqft NaN  
1  2001 · 99 years  $628000   $436 psf  1442 sqft NaN  
2  2005 · 99 years  $498000   $421 psf  1184 sqft NaN  
3          2003 ·   $410000   $343 psf  1195 sqft NaN  
4  2015 · 99 years  $650000  $5804 psf   112 sqft NaN  
0      Singapore 532475 · D19
1      Singapore 541298 · D19
2      Singapore 822641 · D19
3      Singapore 820135 · D19
4      Singapore 823312 · D19
                ...          
135    Singapore 550144 · D19
136

In [11]:
# Retrieve Postal Code of apartments using regex 
regex_pc = re.compile(r'\d{6}')
latitudes,longitudes = [],[]
for row in range(df.shape[0]):
    district = df[" District"][row]
    pc = regex_pc.search(district).group()
    lat, lon = latlon(pc)
    latitudes.append(lat)
    longitudes.append(lon)
# Actually Postal Code column is redundant, you should consider removing it 
# df["Postal Code"] = pc 
df["Latitude"]=latitudes 
df["Longitude"]=longitudes

In [12]:
# https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

def basemap(default_location=[1.375584516, 103.8835163], default_zoom_start=13):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

base_map = basemap()
base_map

NameError: name 'folium' is not defined

In [ ]:
# Temporary -- remove $ sign from cost column for input 
df["Price"] = df[" Cost"].apply(lambda x:x.replace("$","")).astype(int)

In [ ]:
d = df[['Latitude','Longitude','Price']].values.tolist()
print(d[:5])

In [ ]:
gradient = {.3: 'blue', .65: 'lime', 1: 'red'}
max_amount = df['Price'].max()

In [ ]:
m = HeatMap(d,radius=10,min_opacity=0.1, max_zoom=1, blur=10,max_val=max_amount, gradient=gradient).add_to(base_map)

base_map

CHECK2

In [24]:
import pandas as pd

In [25]:
df = pd.read_csv("flats04_Oct.csv")

In [26]:
df = df.drop_duplicates()

In [27]:
df["ft2"] = df["ft2"].apply(lambda x: x.split()[0])

In [41]:
df["ft2"] = pd.to_numeric(df["ft2"])
df["Psf"] = df["Cost"]/df["ft2"]

In [51]:
df.sort_values(by = 'Psf', ascending = True).head(50)

,Address,District,Rooms,Lease,Cost,Psf,ft2
64,524 Serangoon Nth Ave 4 · D19,None,None,None,388000,346.738159,1119
40,236 Compassvale Walk · D19,None,None,None,460000,347.694633,1323
34,135 Lor Ah Soo · D19,None,None,None,399000,349.693252,1141
59,156 Hougang St 11 · D19,None,None,None,485000,357.669617,1356
138,553 Serangoon Nth Ave 3 · D19,None,None,None,400000,367.985281,1087
158,132 Edgedale Plains · D19,None,None,None,440000,368.509213,1194
84,307D Anchorvale Rd · D19,None,None,None,440000,371.621622,1184
129,135 Lor Ah Soo · D19,None,None,None,535000,376.760563,1420
69,157B Rivervale Cres · D19,None,None,None,450000,376.884422,1194
82,158A Rivervale Cres · D19,None,None,None,450000,380.067568,1184


In [34]:
390000-90000

488000

In [49]:
99-35 + 25

89